In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("version2.csv")
df.head()

,Unnamed: 0,company,company_public_response,company_response,complaint_id,narratives,consumer_consent_provided,consumer_disputed,date_received,date_sent_to_company,...,state,sub_issue,sub_product,submitted_via,tags,timely,zip_code,Qtr,product_new,label
0,0,Experian Information Solutions Inc.,Company has responded to the consumer and the ...,Closed with explanation,2788944,There are many mistakes appear in my report wi...,Consent provided,NaN,2018-01-20,2018-01-20,...,MD,Account information incorrect,Credit reporting,Web,NaN,Yes,212XX,2018Q1,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
1,1,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Company has responded to the consumer and the ...,Closed with explanation,2773990,I was pulling my credit to looking into buying...,Consent provided,NaN,2018-01-06,2018-01-06,...,FL,Information belongs to someone else,Credit reporting,Web,NaN,Yes,347XX,2018Q1,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
2,2,"Caliber Home Loans, Inc.",NaN,Closed with explanation,2737995,"Since XXXX XXXX, I was working with a loan con...",Consent provided,NaN,2017-11-26,2017-11-26,...,MD,NaN,Conventional home mortgage,Web,NaN,Yes,208XX,2017Q4,Mortgage,"('Mortgage', 'apply for mortgage')"
3,3,"Select Management Resources, LLC",Company believes it acted appropriately as aut...,Closed with non-monetary relief,2674806,On XX/XX/XXXX I went to NTL in Delaware for a ...,Consent provided,NaN,2017-09-15,2017-09-15,...,PA,NaN,Title loan,Web,NaN,Yes,190XX,2017Q3,Loan,"('loan', 'payment')"
4,4,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Company has responded to the consumer and the ...,Closed with non-monetary relief,2683460,"Hello CFPB, I am sending this complaint to you...",Consent provided,NaN,2017-09-25,2017-09-25,...,MD,Their investigation did not fix an error on yo...,Credit reporting,Web,NaN,Yes,212XX,2017Q3,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."


In [3]:
df.shape

(87592, 22)

In [5]:
text = df['narratives'].tolist()

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
#tfidf_vect = TfidfVectorizer(tokenizer = get_doc_tokens)
tfidf_vect = TfidfVectorizer(stop_words = "english")
dtm = tfidf_vect.fit_transform(text)

In [8]:
dtm.shape

(87592, 53784)

In [13]:
label = df['label'].tolist()
len(label)

87592

In [10]:
all_label = list(set(label))

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(dtm, label,\
                                    test_size = 0.3, stratify = label,\
                                                    random_state = 123)

In [82]:
from sklearn.model_selection import cross_validate
from sklearn import svm 

metrics = ['precision_macro','recall_macro','f1_macro']
clf = svm.LinearSVC().fit(X_train, y_train)
predicted = clf.predict(X_test)
precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, labels = all_label)
print(classification_report(y_test, predicted, target_names = all_label))

                                                                                                                                                                      precision    recall  f1-score   support

                                                                                          ('Debt collection', 'Took or threatened to take negative or legal action')       0.47      0.36      0.41       198
                                                                                           ('Checking or savings account', 'Problem caused by your funds being low')       0.51      0.64      0.57       848
                                                                                                              ('Debt collection', 'Written notification about debt')       0.57      0.50      0.53       181
                                                                          ('Credit reporting, credit repair services, or other personal consumer reports', 'others')       0.53

In [117]:
predicted[1129]

"('Money transfer, virtual currency, or money service', 'service')"

In [108]:
y_t = pd.DataFrame(y_test)
len(y_t[y_t[0]=="('Money transfer, virtual currency, or money service', 'service')"])

59

In [118]:
df.iloc[1129,]

Unnamed: 0                                                                1129
company                                    Experian Information Solutions Inc.
company_public_response      Company has responded to the consumer and the ...
company_response                                       Closed with explanation
complaint_id                                                           2725704
narratives                   i am not sure of the balances but before i tur...
consumer_consent_provided                                     Consent provided
consumer_disputed                                                          NaN
date_received                                                       2017-11-10
date_sent_to_company                                                2017-11-10
issue                                     Incorrect information on your report
product                      Credit reporting, credit repair services, or o...
state                                               

#### Predict new doc

In [120]:
new_doc =[]
new_doc.append(str(input("input: ")))
X_new_tfidf = tfidf_vect.transform(list(new_doc))
#X_new_tfidf.shape
pred_new = clf.predict(X_new_tfidf)
pred_new

input: I made a   XXXX   dollar deposit to my sister 's bank account in   XXXX   on   XXXX   XXXX  , 2017 but when I contacted her to inform her she told me that that bank account had been closed on   XXXX   XXXX  , 2017. She went to the bank and tried to reopen the account to collect the money and they said that it was not possible to do that the money had been returned to whoever sent it and now neither the bank in  XXXX  nor the company I was with whom the sending was made They want to make them responsible for delivering the money. The company vigo says that in its system appears the money paid and that expect a response from the bank but have spent 2 weeks and do not give me any solution to the problem if not to keep waiting. I thank you in advance for the help you can give me.


array(["('Checking or savings account', 'Closing an account')"], 
      dtype='<U164')

In [22]:
df[df['label']=="('Checking or savings account', 'Managing an account')"]['narratives'].tolist()[3] 

"I lodged a complaint few days back to CFPB. The case # is XXXX. I opened a checking account online with {$1000.00} as initial deposit on XXXX XXXX and made XXXX direct deposits by XXXX. The bank had advertised that if I made XXXX direct deposits it will credit my account {$200.00}. The bank has not yet deposited the amount to my account. \nI filed a complaint through CFPB and the company responded that my account did n't have a promotion code attached to the account. The problem is at the bank 's end. Their site through which I opened the account stated, accounts opened online will have the code automatically attached. I have attached the screen shot as evidence. \n\nWhat I find frustrating is, I am not the only XXXX. The bank is trying to give excuses not to give the bonus. I believe it is a fraudulent business practice."

In [115]:
zero = [('Checking or savings account', 'Opening an account'),('Money transfer, virtual currency, or money service', 'service')]
pre = pd.DataFrame(predicted)
idx = pre[pre[0]=="('Money transfer, virtual currency, or money service', 'service')"].index
df.iloc[idx,]['narratives'].tolist()

["i am not sure of the balances but before i turned XXXX on XX/XX/XXXX my mother discovered someone used my information she contacted XXXX XXXX and XXXX XXXX they closed the account due to fraud. she than contacted experian credit bureau to make sure the accounts were taken off of my credit file. They had her submit multiple documents back in XX/XX/XXXX. The accounts still were n't removed when I turned XXXX in XXXX I contacted them again they made me submit the same documents in XXXX of XX/XX/XXXXand the accounts still remain on my credit file. The customer service rep even attacked me like it was my fault some used my information fraudulently. I do n't understand if the company closed the account due to fraud why experian as not deleted the information",
 "In  XXXX   XXXX  I purchased a  XXXX   XXXX   XXXX . In  XXXX   XXXX  the car was repossessed for delinquent payments. I paid {$980.00} to recover the vehicle and deferred the current payment (  XXXX   XXXX   )  to remain in a good

In [113]:
aa = pd.DataFrame()
aa['idx'] = idx
aa['pred']= predicted[idx]
aa['true']=np.array(df.iloc[idx,-1])
aa['true']

,idx,pred,true
0,42,"('Checking or savings account', 'Opening an ac...","('Credit reporting, credit repair services, or..."
1,279,"('Checking or savings account', 'Opening an ac...","('loan', 'Managing the loan or lease')"
2,374,"('Checking or savings account', 'Opening an ac...","('Credit reporting, credit repair services, or..."
3,506,"('Checking or savings account', 'Opening an ac...","('Credit reporting, credit repair services, or..."
4,1197,"('Checking or savings account', 'Opening an ac...","('Debt collection', 'Communication tactics')"
5,1270,"('Checking or savings account', 'Opening an ac...","('Debt collection', 'Written notification abou..."
6,1450,"('Checking or savings account', 'Opening an ac...","('Mortgage', 'Struggling to pay mortgage')"
7,1581,"('Checking or savings account', 'Opening an ac...","('Mortgage', 'apply for mortgage')"
8,1951,"('Checking or savings account', 'Opening an ac...","('Mortgage', 'Trouble during payment process')"
9,2351,"('Checking or savings account', 'Opening an ac...","('Debt collection', 'Attempts to collect debt ..."


### TruncatedSVD 

In [12]:
#Try TruncatedSVD to reduce the dimension
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
svd = TruncatedSVD(n_components = 5000).fit(dtm.T)

In [16]:
#Display the reduced dataset
svd_df = pd.DataFrame(svd.components_.T)
svd_df

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.001475,0.002359,-0.003361,0.002444,-0.000020,0.001234,-0.002623,0.001314,-0.002601,-0.002440,...,-0.000592,0.000435,0.000743,2.584052e-04,0.000439,-0.000209,0.000328,0.000940,-0.000063,2.429146e-05
1,0.002783,0.003680,-0.008802,-0.002922,0.001408,0.000677,-0.001434,0.003375,0.007556,0.000841,...,0.000085,0.000196,-0.000353,-1.317590e-04,-0.000877,-0.000205,-0.000207,0.000222,0.001289,-1.337258e-04
2,0.003592,-0.004122,-0.000030,-0.001300,-0.000850,0.001331,-0.005462,0.002129,0.002254,-0.001756,...,0.000243,0.001501,-0.000895,4.564632e-04,-0.002061,-0.001232,-0.002689,-0.000206,0.001568,8.541620e-04
3,0.001768,-0.004007,0.001368,0.001730,-0.001341,-0.001410,-0.002548,0.003922,0.000074,0.002194,...,0.000599,0.000202,0.000317,-5.192736e-04,-0.000700,-0.000679,0.000950,0.001673,0.000022,-6.626080e-04
4,0.004287,0.008104,0.001722,0.002487,-0.001347,0.003967,0.001260,-0.005974,0.003513,-0.001873,...,0.000117,0.000239,-0.000379,1.019438e-04,-0.000297,-0.000221,-0.000756,0.000466,-0.000282,-4.760833e-04
5,0.002595,0.004677,-0.007464,-0.003567,0.001908,0.002305,0.005371,0.010223,0.008610,0.006982,...,-0.001050,-0.000613,0.000980,5.418897e-05,0.001090,-0.000369,-0.000484,0.001450,0.000884,-1.032335e-03
6,0.003218,-0.002453,0.001590,0.001558,-0.003532,0.000941,0.000677,-0.001287,-0.000600,0.006325,...,-0.000608,0.000082,-0.000471,1.275898e-03,0.001949,-0.000539,-0.001115,0.000068,0.001853,-1.258846e-03
7,0.002897,-0.000594,-0.002990,-0.000303,-0.001669,-0.001464,0.001433,-0.002395,0.001661,-0.004150,...,-0.000678,-0.000106,0.000842,-1.580396e-03,0.000065,0.000271,-0.000972,-0.000111,0.000526,-8.773040e-05
8,0.004467,-0.002212,0.000146,0.005556,-0.005551,-0.003878,0.001422,-0.003349,-0.001518,0.003794,...,-0.000903,-0.000811,-0.000888,5.218108e-04,-0.000830,-0.000668,-0.000157,-0.000383,0.000663,-1.257434e-03
9,0.003167,0.001078,0.000964,0.000861,-0.003158,-0.000972,-0.002180,0.001946,0.001185,-0.005482,...,0.000052,0.000254,-0.000892,1.600784e-04,-0.000268,-0.000351,-0.000136,-0.000142,-0.000871,1.065729e-03
